In [1]:
import requests
import re
import json
from bs4 import BeautifulSoup
from selenium import webdriver 
from os.path import exists
from time import sleep

In [2]:
def get_all_inventory(url):
    html = requests.get(url)
    soup = BeautifulSoup(html.text, "html.parser")
    links = []
    boxes = soup.findAll('div', {'class': 'container-2x1-stack'})
    for box in boxes:
        for a in box.findAll('a', {'class': "mb0-5"}, href = True):
            links.append(a['href'])
    links.append('https://www.jellycat.com/us/bags-purses/')
    links.append('https://www.jellycat.com/us/bag-charms/')
    links.append('https://www.jellycat.com/us/backpacks/')
    return links

In [3]:
def get_all_products(url):
    driver = webdriver.Chrome('./chromedriver')
    driver.get(url)
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    product_links = []
    for item in soup.findAll('div', {'class': 'listing gproduct relative'}):
        for a in item.findAll('a', {'data-listing': 'name'}, href = True):
            product_links.append('https://www.jellycat.com' + a['href'])
    return product_links

In [4]:
def check_status(url):
    html = requests.get(url)
    soup = BeautifulSoup(html.text, "html.parser")
    script = soup.findAll('script', type='text/javascript')[11].text
    raw_json = re.search('variants = ({.*}[ \t\r\n\f]*})', script).group(1).replace("\'", "\"")
    site_json = json.loads(raw_json)
    name = site_json[list(site_json.keys())[0]]['name']
    status = site_json[list(site_json.keys())[0]]['lead_text_summary']
    return (name, status)

In [5]:
if not exists('./product_links.txt'):
    inventory_links = get_all_inventory('https://www.jellycat.com/us/site-map/')
    product_links = []
    for inventory_link in inventory_links:
        print(inventory_link)
        product_links += get_all_products(inventory_link)
    with open('product_links.txt', 'x') as f:
        for link in product_links:
            f.write(link + '\n')

In [ ]:
inventory_status = {}
error_items = set()
with open('./product_links.txt', 'r') as f:
    product_links = f.read().splitlines()
    counter, total = 0, len(product_links)
    for link in product_links:
        product_link = link[:-1]
        try:
            name, status = check_status(product_link)
            inventory_status[name] = status
            print(f"Obtained status {counter}/{total}")
            counter += 1
#             sleep(0.1)
        except:
            print(f"Error occurred with: {product_link}")
            error_items.add(product_link)

Obtained status 0/604
Obtained status 1/604
Obtained status 2/604
Obtained status 3/604
Obtained status 4/604
Error occurred with: https://www.jellycat.com/us/bashful-blue-bunny-soother-sob444b
Error occurred with: https://www.jellycat.com/us/bashful-pink-bunny-soother-sob444p
Error occurred with: https://www.jellycat.com/us/bashful-beige-bunny-soother-so4bb
Obtained status 5/604
Obtained status 6/604
Obtained status 7/604
Obtained status 8/604
Obtained status 9/604
Error occurred with: https://www.jellycat.com/us/bashful-silver-bunny-soother-so4bs
Obtained status 10/604
Obtained status 11/604
Obtained status 12/604
Obtained status 13/604
Obtained status 14/604
Error occurred with: https://www.jellycat.com/us/bashful-cream-bunny-soother-bb4bc
Obtained status 15/604
Obtained status 16/604
Obtained status 17/604
Obtained status 18/604
Obtained status 19/604
Obtained status 20/604
Obtained status 21/604
Obtained status 22/604
Obtained status 23/604
Obtained status 24/604
Obtained status 2

Obtained status 153/604
Obtained status 154/604
Obtained status 155/604
Error occurred with: https://www.jellycat.com/us/riverside-rambler-frog-riv3f
Error occurred with: https://www.jellycat.com/us/riverside-rambler-badger-riv3b
Obtained status 156/604
Obtained status 157/604
Error occurred with: https://www.jellycat.com/us/campfire-critter-bear-camp3b
Error occurred with: https://www.jellycat.com/us/campfire-critter-moose-camp3m
Error occurred with: https://www.jellycat.com/us/campfire-critter-raccoon-camp3r
Obtained status 158/604
Error occurred with: https://www.jellycat.com/us/lottie-bunny-ballet-lot3bb
Error occurred with: https://www.jellycat.com/us/lottie-bunny-party-lot3bp
Obtained status 159/604
Obtained status 160/604
Obtained status 161/604
Obtained status 162/604
Obtained status 163/604
Obtained status 164/604
Obtained status 165/604
Obtained status 166/604
Obtained status 167/604
Obtained status 168/604
Obtained status 169/604
Obtained status 170/604
Obtained status 171/6

In [ ]:
csv_str = ''
for key, value in inventory_status.items():
    csv_str += (key + ',' + value + '\n')
csv_str
with open('inventory_status.csv', 'w') as f:
    f.write(csv_str)